In [2]:
import os
import polars as pl

In [3]:
kaggle_run_type = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', None)
if kaggle_run_type:
    DATA_PATH = "/kaggle/input/linking-writing-processes-to-writing-quality"
else:
    DATA_PATH = "../../data"

In [4]:
# calculate the average score on the training set
avg_score = (
    pl.scan_csv(f"{DATA_PATH}/train_scores.csv")
    .select(pl.col("score").mean())
    .collect()
    .item()
)
# create a submission with the average score for every essay
submission = (
    pl.scan_csv(f"{DATA_PATH}/test_logs.csv")
    .select(pl.col("id").unique(), pl.lit(avg_score).alias("score"))
    .collect()
)

In [5]:
# expected RMSE for this submission
(
    pl.scan_csv(f"{DATA_PATH}/train_scores.csv")
    .select(
        (pl.col("score") - pl.col("score").mean()).pow(2).mean().sqrt()
    )
    .collect()
    .item()
)

1.0247300748982624

In [27]:
# write the submission file
submission.write_csv("submission.csv")